<a href="https://colab.research.google.com/github/etappau/ProjectTerm/blob/master/TermProject_teamPaulAbdullaMohanned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
train0 = pd.read_csv('AskAPatient.fold-0.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train1 = pd.read_csv('AskAPatient.fold-1.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train2 = pd.read_csv('AskAPatient.fold-2.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train3 = pd.read_csv('AskAPatient.fold-3.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train4 = pd.read_csv('AskAPatient.fold-4.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train5 = pd.read_csv('AskAPatient.fold-5.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train6 = pd.read_csv('AskAPatient.fold-6.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train7 = pd.read_csv('AskAPatient.fold-7.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train8 = pd.read_csv('AskAPatient.fold-8.train.txt', encoding='ISO-8859-1', sep='\t', header = None)
train9 = pd.read_csv('AskAPatient.fold-9.train.txt', encoding='ISO-8859-1', sep='\t', header = None)

In [0]:
frames = [train0, train1, train2, train3, train4, train5, train6, train7, train8, train9]

In [0]:
train = pd.concat(frames)

In [0]:
train.columns = ['CUID', 'Disease', 'Patient Complaint Narrative']

In [0]:
#train = train.drop_duplicates(subset=['CUID', 'Disease', 'Patient Complaint Narrative'],keep='first')

In [55]:
train.shape

(156652, 3)

In [0]:
valid0 = pd.read_csv('AskAPatient.fold-0.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid1 = pd.read_csv('AskAPatient.fold-1.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid2 = pd.read_csv('AskAPatient.fold-2.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid3 = pd.read_csv('AskAPatient.fold-3.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid4 = pd.read_csv('AskAPatient.fold-4.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid5 = pd.read_csv('AskAPatient.fold-5.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid6 = pd.read_csv('AskAPatient.fold-6.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid7 = pd.read_csv('AskAPatient.fold-7.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid8 = pd.read_csv('AskAPatient.fold-8.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)
valid9 = pd.read_csv('AskAPatient.fold-9.validation.txt', encoding='ISO-8859-1', sep='\t', header = None)

In [0]:
testframes = [valid0, valid1, valid2, valid3, valid4, valid5, valid6, valid7, valid8, valid9]

In [0]:
test = pd.concat(testframes)

In [0]:
test.columns = ['CUID', 'Disease', 'Patient Complaint Narrative']

In [0]:
#test = test.drop_duplicates(subset=['CUID', 'Disease', 'Patient Complaint Narrative'],keep='first')

In [60]:
test.shape

(7926, 3)

In [0]:
import re
import string

In [62]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords

In [0]:
stop_words = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in stop_words) # remove stopwors from text
    return text

In [0]:
train['Patient Complaint Narrative'] = train['Patient Complaint Narrative'].apply(clean_text)
train['Patient Complaint Narrative'] = train['Patient Complaint Narrative'].str.replace('\d+', '')
train['Disease'] = train['Disease'].apply(clean_text)
train['Disease'] = train['Disease'].str.replace('\d+', '')

In [0]:
test['Patient Complaint Narrative'] = test['Patient Complaint Narrative'].apply(clean_text)
test['Patient Complaint Narrative'] = test['Patient Complaint Narrative'].str.replace('\d+', '')
test['Disease'] = test['Disease'].apply(clean_text)
test['Disease'] = test['Disease'].str.replace('\d+', '')

In [0]:
train.CUID.value_counts()

In [0]:
test.Disease.value_counts()

In [0]:
from collections import Counter

In [0]:
Counter(train["CUID"])

In [0]:
test.groupby('CUID')['Patient Complaint Narrative'].count()

In [69]:
!pip install bert-tensorflow
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df
def create_tokenizer_from_hub_module(bert_model_hub):
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(bert_model_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def make_features(dataset, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      bert_model_hub,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
#         f1_score = tf.contrib.metrics.f1_score(
#             label_ids,
#             predicted_labels)
#         auc = tf.metrics.auc(
#             label_ids,
#             predicted_labels)
#         recall = tf.metrics.recall(
#             label_ids,
#             predicted_labels)
#         precision = tf.metrics.precision(
#             label_ids,
#             predicted_labels) 
#         true_pos = tf.metrics.true_positives(
#             label_ids,
#             predicted_labels)
#         true_neg = tf.metrics.true_negatives(
#             label_ids,
#             predicted_labels)   
#         false_pos = tf.metrics.false_positives(
#             label_ids,
#             predicted_labels)  
#         false_neg = tf.metrics.false_negatives(
#             label_ids,
#             predicted_labels)
        return {
            "eval_accuracy": accuracy
#             "f1_score": f1_score,
#             "auc": auc,
#             "precision": precision,
#             "recall": recall,
#             "true_positives": true_pos,
#             "true_negatives": true_neg,
#             "false_positives": false_pos,
#             "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE, num_train_steps, num_warmup_steps, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_model_hub = bert_model_hub,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config

def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
               MAX_SEQ_LENGTH = 128,
              BATCH_SIZE = 32,
              LEARNING_RATE = 2e-5,
              NUM_TRAIN_EPOCHS = 3.0,
              WARMUP_PROPORTION = 0.1,
              SAVE_SUMMARY_STEPS = 100,
               SAVE_CHECKPOINTS_STEPS = 10000,
              bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub, 
                                  OUTPUT_DIR, 
                                  SAVE_SUMMARY_STEPS, 
                                  SAVE_CHECKPOINTS_STEPS, 
                                  label_list, 
                                  LEARNING_RATE, 
                                  num_train_steps, 
                                  num_warmup_steps, 
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator
    
import random
random.seed(10)
OUTPUT_DIR = 'output'

In [0]:
#from sklearn.model_selection import train_test_split

In [0]:
#train, test = train_test_split(df, test_size=0.2, random_state=1)

In [29]:
len(train)

5054

In [30]:
len(test)

2749

In [0]:
train = train.sample(len(train))

In [0]:
train.head()

In [0]:
test.head()

In [0]:
myparam = {
    "DATA_COLUMN": "Patient Complaint Narrative",
    "LABEL_COLUMN": "CUID",
    "LEARNING_RATE": 2e-5,
    "NUM_TRAIN_EPOCHS": 2
}

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [72]:
result, estimator = run_on_dfs(train, test, **myparam)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0423 18:27:50.192259 140132974561152 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 156652


I0423 18:27:54.182264 140132974561152 run_classifier.py:774] Writing example 0 of 156652


INFO:tensorflow:*** Example ***


...

I0423 18:30:14.769600 140132974561152 evaluation.py:277] Finished evaluation at 2019-04-23-18:30:14


INFO:tensorflow:Saving dict for global step 15793: eval_accuracy = 0.91862226, global_step = 15793, loss = 0.9915121


I0423 18:30:14.771738 140132974561152 estimator.py:1979] Saving dict for global step 15793: eval_accuracy = 0.91862226, global_step = 15793, loss = 0.9915121


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15793: output/model.ckpt-15793


I0423 18:30:14.779061 140132974561152 estimator.py:2039] Saving 'checkpoint_path' summary for global step 15793: output/model.ckpt-15793


In [73]:
result

{'eval_accuracy': 0.91862226, 'global_step': 15793, 'loss': 0.9915121}